In [ ]:
import pandas as pd
import requests
import io

def download_premier_league_safe():    
    all_dataframes = []
    
    for start_year in range(2000, 2025):
        end_year = start_year + 1
        year_code = f"{str(start_year)[2:]}{str(end_year)[2:]}"
        season_name = f"{start_year}-{end_year}"
        url = f"https://www.football-data.co.uk/mmz4281/{year_code}/E0.csv"
        
        try:
            response = requests.get(url, timeout=10)
            
            if response.status_code == 200:
                content = response.content
                df = pd.read_csv(io.BytesIO(content), encoding='latin-1', 
                                on_bad_lines='skip', engine='python')
                df = df.dropna(how='all')
                df['Season'] = season_name 
                all_dataframes.append(df)
                print(f"{season_name:10} | {len(df):3} матчей | {len(df.columns):3} столбцов")
            else:
                print(f"{season_name:10} | HTTP {response.status_code}")
                
        except Exception as e:
            print(f"{season_name:10} | {str(e)[:50]}")
    
    if not all_dataframes:
        return None

    common_columns = set(all_dataframes[0].columns)
    
    for df_season in all_dataframes[1:]:
        common_columns &= set(df_season.columns)
    
    common_columns = sorted(list(common_columns))
    
    print(f"Общих столбцов: {len(common_columns)}")
    print(f"Список: {common_columns}\n")

    filtered_dataframes = [df[common_columns] for df in all_dataframes]
    
    combined = pd.concat(filtered_dataframes, ignore_index=True, sort=False)
    
    print(f"Загружено: {len(all_dataframes)} сезонов")
    print(f"Всего матчей: {len(combined)}")
    print(f"Итоговых столбцов: {len(combined.columns)}")
    
    return combined

df = download_premier_league_safe()

if df is not None:
    df.to_csv('premier_league_complete.csv', index=False)
    print(f"Форма: {df.shape}")
    print(f"Итоговые столбцы:\n{df.columns.tolist()}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress, pearsonr

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

df['TotalGoals'] = df['FTHG'] + df['FTAG']

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

ax1 = fig.add_subplot(gs[0, 0])
max_goals = int(max(df['FTHG'].max(), df['FTAG'].max())) + 2
bins_goals = range(0, max_goals)
ax1.hist([df['FTHG'], df['FTAG']], bins=bins_goals, alpha=0.6, 
         label=['Хозяева', 'Гости'], edgecolor='black', color=['#2E86AB', '#A23B72'])
ax1.set_xlabel('Количество голов', fontsize=11, fontweight='bold')
ax1.set_ylabel('Частота', fontsize=11, fontweight='bold')
ax1.set_title('Распределение голов', fontsize=13, fontweight='bold', pad=15)
ax1.legend(fontsize=10, framealpha=0.9)
ax1.grid(alpha=0.3, linestyle='--')

mean_home = df['FTHG'].mean()
mean_away = df['FTAG'].mean()
ax1.axvline(mean_home, color='#2E86AB', linestyle='--', linewidth=2, alpha=0.7)
ax1.axvline(mean_away, color='#A23B72', linestyle='--', linewidth=2, alpha=0.7)

ax2 = fig.add_subplot(gs[0, 1])
max_total_goals = int(df['TotalGoals'].max()) + 2
bins_total = range(0, max_total_goals)
ax2.hist(df['TotalGoals'], bins=bins_total, color='#F18F01', alpha=0.75, 
         edgecolor='black', linewidth=1.2)
ax2.axvline(df['TotalGoals'].mean(), color='#C73E1D', linestyle='--', 
            linewidth=2.5, label=f"Среднее: {df['TotalGoals'].mean():.2f}")
ax2.axvline(df['TotalGoals'].median(), color='#6A994E', linestyle=':', 
            linewidth=2.5, label=f"Медиана: {df['TotalGoals'].median():.1f}")
ax2.set_xlabel('Общее количество голов', fontsize=11, fontweight='bold')
ax2.set_ylabel('Частота', fontsize=11, fontweight='bold')
ax2.set_title('Распределение общих голов за матч', fontsize=13, fontweight='bold', pad=15)
ax2.legend(fontsize=10, framealpha=0.9)
ax2.grid(alpha=0.3, linestyle='--')

ax3 = fig.add_subplot(gs[0, 2])
ftr_counts = df['FTR'].value_counts()
ftr_pct = (ftr_counts / len(df) * 100).round(1)

colors_ftr = {'H': '#06A77D', 'D': '#95A3A4', 'A': '#F39237'}
labels_map = {'H': 'Хозяева', 'D': 'Ничья', 'A': 'Гости'}

bars = ax3.bar(range(len(ftr_counts)), ftr_counts.values, 
               color=[colors_ftr.get(x, '#95A3A4') for x in ftr_counts.index], 
               edgecolor='black', linewidth=1.5, alpha=0.85)

ax3.set_xlabel('Результат', fontsize=11, fontweight='bold')
ax3.set_ylabel('Количество матчей', fontsize=11, fontweight='bold')
ax3.set_title('Распределение результатов', fontsize=13, fontweight='bold', pad=15)
ax3.set_xticks(range(len(ftr_counts)))
ax3.set_xticklabels([labels_map.get(x, x) for x in ftr_counts.index], fontsize=10)
ax3.grid(alpha=0.3, axis='y', linestyle='--')

for idx, bar in enumerate(bars):
    height = bar.get_height()
    result_code = ftr_counts.index[idx]
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}\n({ftr_pct[result_code]:.1f}%)',
             ha='center', va='bottom', fontsize=9, fontweight='bold')

ax4 = fig.add_subplot(gs[1, 0])
max_shots = int(max(df['HS'].max(), df['AS'].max())) + 2
bins_shots = range(0, max_shots, 2)
ax4.hist([df['HS'], df['AS']], bins=bins_shots, alpha=0.6, 
         label=['Хозяева', 'Гости'], edgecolor='black', color=['#1982C4', '#FF6B6B'])
ax4.set_xlabel('Количество ударов', fontsize=11, fontweight='bold')
ax4.set_ylabel('Частота', fontsize=11, fontweight='bold')
ax4.set_title('Распределение ударов', fontsize=13, fontweight='bold', pad=15)
ax4.legend(fontsize=10, framealpha=0.9)
ax4.grid(alpha=0.3, linestyle='--')

ax5 = fig.add_subplot(gs[1, 1])
ax5.scatter(df['HST'], df['FTHG'], alpha=0.4, s=20, c='#06A77D', 
            label='Хозяева', edgecolors='none')
ax5.scatter(df['AST'], df['FTAG'], alpha=0.4, s=20, c='#F39237', 
            label='Гости', edgecolors='none')

ax5.set_xlabel('Удары в створ', fontsize=11, fontweight='bold')
ax5.set_ylabel('Голы', fontsize=11, fontweight='bold')
ax5.set_title('Корреляция: удары в створ → голы', fontsize=13, fontweight='bold', pad=15)
ax5.legend(fontsize=10, framealpha=0.9, loc='upper left')
ax5.grid(alpha=0.3, linestyle='--')

# Линии тренда
valid_home = df[['HST', 'FTHG']].dropna()
if len(valid_home) > 0:
    slope_h, intercept_h, r_h, _, _ = linregress(valid_home['HST'], valid_home['FTHG'])
    x_h = np.linspace(valid_home['HST'].min(), valid_home['HST'].max(), 100)
    y_h = slope_h * x_h + intercept_h
    ax5.plot(x_h, y_h, color='#06A77D', linestyle='--', linewidth=2.5, alpha=0.8)

valid_away = df[['AST', 'FTAG']].dropna()
if len(valid_away) > 0:
    slope_a, intercept_a, r_a, _, _ = linregress(valid_away['AST'], valid_away['FTAG'])
    x_a = np.linspace(valid_away['AST'].min(), valid_away['AST'].max(), 100)
    y_a = slope_a * x_a + intercept_a
    ax5.plot(x_a, y_a, color='#F39237', linestyle='--', linewidth=2.5, alpha=0.8)

ax6 = fig.add_subplot(gs[1, 2])
season_goals = df.groupby('Season').agg({
    'TotalGoals': 'mean',
    'FTHG': 'mean',
    'FTAG': 'mean'
}).sort_index()

x_pos = range(len(season_goals))
ax6.plot(x_pos, season_goals['TotalGoals'].values, marker='o', linewidth=2.5, 
         markersize=7, color='#6A4C93', label='Всего голов', alpha=0.9)
ax6.plot(x_pos, season_goals['FTHG'].values, marker='s', linewidth=1.5, 
         markersize=5, color='#06A77D', label='Хозяева', alpha=0.7, linestyle='--')
ax6.plot(x_pos, season_goals['FTAG'].values, marker='^', linewidth=1.5, 
         markersize=5, color='#F39237', label='Гости', alpha=0.7, linestyle='--')

overall_mean = df['TotalGoals'].mean()
ax6.axhline(overall_mean, color='#C73E1D', linestyle=':', linewidth=2, 
            label=f'Общее среднее: {overall_mean:.2f}', alpha=0.8)

ax6.set_xlabel('Сезон', fontsize=11, fontweight='bold')
ax6.set_ylabel('Среднее количество голов за матч', fontsize=11, fontweight='bold')
ax6.set_title('Динамика результативности по сезонам', fontsize=13, fontweight='bold', pad=15)
ax6.set_xticks(range(0, len(season_goals), 2))
ax6.set_xticklabels(season_goals.index[::2], rotation=45, ha='right', fontsize=8)
ax6.legend(fontsize=9, framealpha=0.9, loc='best')
ax6.grid(alpha=0.3, linestyle='--')

# Выделение выбросов
max_season = season_goals['TotalGoals'].idxmax()
min_season = season_goals['TotalGoals'].idxmin()
max_idx = list(season_goals.index).index(max_season)
min_idx = list(season_goals.index).index(min_season)

ax6.scatter([max_idx], [season_goals['TotalGoals'].iloc[max_idx]], 
            s=200, c='red', marker='*', zorder=5, edgecolors='black', linewidth=1.5)
ax6.scatter([min_idx], [season_goals['TotalGoals'].iloc[min_idx]], 
            s=200, c='blue', marker='*', zorder=5, edgecolors='black', linewidth=1.5)

fig.suptitle('Анализ выбросов: Premier League (2000-2025)', 
             fontsize=16, fontweight='bold', y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig('outliers_analysis_final.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
